In [1]:
import csv
import numpy as np
import pandas as pd
import operator
from collections import defaultdict
from scipy.sparse import csr_matrix
import gensim.models.word2vec as w2v
import gensim.models
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
data_path = '../data/mimic3_new/'

In [3]:
df = pd.read_csv(data_path + 'all_50.csv')

In [4]:
df.head()

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE,ICD10_CODE
0,17,161087,admission date date birth sex f service cardio...,311;88.72;38.93;511.9;272.4,F32.9;B244YZZ;02H633Z;J91.8;E78.4
1,20,157681,admission date date birth sex f service histor...,39.61;36.15;250.00;414.01;401.9;272.4,5A1221Z;0210088;E11.9;I25.10;I10;E78.4
2,33,176176,admission date service history present illness...,038.9,A41.9
3,38,185910,admission date date birth sex service urology ...,V45.81;995.92;38.93;428.0;96.6;584.9;96.72;427...,Z95.1;R65.20;02H633Z;I50.814;0DH67UZ;N17.9;5A1...
4,41,101757,admission date date birth sex service medicine...,305.1;99.04;38.93;33.24;96.6;496;285.9;507.0;4...,F17.200;30233N1;02H633Z;0B933ZX;0DH67UZ;J44.9;...


### Build Vocab

In [7]:
types = set()
for row in df.itertuples():
    for w in row[-3].split(' '):
        types.add(w)

In [8]:
len(types)

66320

In [9]:
vocab_list = list(types)

In [10]:
vocab_list.sort()

In [11]:
with open(data_path + 'vocab.csv', 'w') as vocab_file:
    for word in vocab_list:
        vocab_file.write(word + "\n")

### Pre-train word embeddings

In [12]:
sentences = df.progress_apply(lambda row: str(row['TEXT']).split(), axis=1)

  0%|          | 0/11368 [00:00<?, ?it/s]

In [13]:
model = w2v.Word2Vec(vector_size=100, min_count=0, workers=4, epochs=50)
model.build_vocab(sentences)

In [14]:
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

(560155531, 599653650)

In [15]:
model.wv.most_similar('fever', topn=10)

[('fevers', 0.8942907452583313),
 ('fivers', 0.6758853793144226),
 ('temperatures', 0.6484113335609436),
 ('temps', 0.6428983211517334),
 ('temperature', 0.6394485831260681),
 ('yestderday', 0.6168003082275391),
 ('febrile', 0.5922038555145264),
 ('addominal', 0.578851580619812),
 ('rflank', 0.5752633213996887),
 ('nonprod', 0.5679307579994202)]

In [16]:
model.save(data_path + 'processed_50.w2v')

### Map words with its embeddings

In [17]:
extract_wvs.gensim_to_embeddings('%s/processed_full.w2v' % MIMIC_3_DIR, '%s/vocab.csv' % MIMIC_3_DIR, Y)

NameError: name 'extract_wvs' is not defined

In [18]:
model = gensim.models.Word2Vec.load(data_path + 'processed_50.w2v')
wv = model.wv                             

In [19]:
vocabdf = pd.read_csv(data_path+'vocab.csv',names=['word'],header=None)

In [20]:
vocab = set(vocabdf['word'])

In [21]:
ind2w = {i+1:w for i,w in enumerate(sorted(vocab))}

In [22]:
PAD_CHAR = "**PAD**"
W = np.zeros((len(ind2w)+1, len(wv.get_vector(wv.index_to_key[0])) ))
words = [PAD_CHAR]
W[0][:] = np.zeros(len(wv.get_vector(wv.index_to_key[0])))
for idx, word in tqdm(ind2w.items()):
    if idx >= W.shape[0]:
        break    
    W[idx][:] = wv.get_vector(word)
    words.append(word)

  0%|          | 0/66320 [00:00<?, ?it/s]

In [23]:
W.shape[0]

66321

In [24]:
with open(data_path + 'processed_50.embed', 'w') as o:
    #pad token already included
    for i in range(len(words)):
        line = [words[i]]
        line.extend([str(d) for d in W[i]])
        o.write(" ".join(line) + "\n")